---
format: 
    html: 
        embed-resources: true
---

# Homework: Markov chains

**Author:** Brian Kwon

* `IMPORTANT`: We have decided to "shake it up" and introduce a `python` assignment, as a break from `R`. 
* Therefore, this assignment MUST be done in `python`, NOT `R`

**Instructions**

* Read and work through all tutorial content and do all exercises below
  
**Submission:**

* You need to upload ONE document to Canvas when you are done
  * (1) An HTML (or PDF) version of the completed form of this notebook 
* The final uploaded version should NOT have any code-errors present 
* All outputs must be visible in the uploaded version, including code-cell outputs, images, graphs, etc

# Introduction 
`Starter code`

## Import and parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# PARAM SET
m=3
pu=0.2; pd=0.3; pl=0.25; pr=0.25
p=[pu,pd,pl,pr]
q=[0,0,0,0,1,0,0,0,0] #initial state probablities

## Overview 

* Markov chains (MC) are important for understanding Markov Decision Processes, even though MC do not involve any decision-making. 
* MCs are mathematical models describing particular types of stochastic processes. 
* **Markov property:**  The next state only depends on the current state, NOT past states 
  * Denote the state at time $t$ by $s_t$
$$p\left(s_{t+1} \mid s_t, s_{t-1}, s_{t-2}, \ldots, s_0\right)=p\left(s_{t+1} \mid s_t\right)$$ 
  * i.e. regardless of the past trajectory, the next state only depends on the current state
* **Markov chain:** A stochastic process where **ALL** states & times have the Markov property. 

## Example: The broken robot
* Consider a robot which is stuck in a $m \times m$ grid world and can only make random moves.
* At EVERY step, the probability for up, down, left, and right is $\mathbf p=(0.2, 0.3, 0.25, 0.25)$
* You can think of $\mathbf p=(0.2, 0.3, 0.25, 0.25)$ as the "rules of the game"
* Assume that if the robot hits the wall, it bounces off and remains in the current state. However it's probabilities for the next steps are still the same as before.
* Does the process have the Markov property? ... Yes! Since, the probability of the next transition depends only on the current state.   

![](image.png)

In [2]:
0.25*0.25

0.0625

##  Transition matrix

* **Initial probability distribution**: $\mathbf q_0$ determines states the system starts in
  * Discrete state set $\rightarrow$ vector containing initialization probabilities for each state 
* **Transition probability matrix $P$** : Each row represents the probability of transition from an arbitrary state to all other possible states (extremely important MC quantity)
- A process with $N$ possible states will result in a $N \times N$ matrix
$$
\begin{bmatrix} P_{1,1} & \cdots & P_{1,n} \\ \vdots & \ddots & \vdots \\ P_{n,1} & \cdots & P_{n,n} \end{bmatrix}
$$
- The probability of transitioning from state $1$ to state $n$ is $P_{1,n}$
- The probability of transitioning from state $n$ to state $n$ is $P_{n,n}$
- This pattern holds for any entry in the matrix: $s_2$ $\rightarrow$ $s_3$ would be $P_{2,3}$ (row 2 column 3)
- Mathematically defined as $P_{ij} = P(s_{t+1} = j, s_t = i)$

## Ergodicity

Time Averages vs. Ensemble Averages: 

* In many situations, you might be interested in understanding the behavior of a system over a long time (time averages) 
* or the behavior of multiple identical systems considered as a group (ensemble averages). 
* Ergodicity implies that these two types of averages will be equivalent, given enough time.

Ergodicity refers to a property of a system or process in which time averages of a single system converge to ensemble averages of a collection of identical systems. It assumes that the system explores its entire state space over time, enabling meaningful equivalence between individual system behavior and group behavior.

## Transition probability matrix

* The following code can be used to compute the transition matrix, for the broken robot problem of arbitrary grid size.
* This was used to compute the matrices from the previous examples

In [3]:
#SOURCE: 2020: Mastering reinforcement learning with python, enes bilgin (1st edition).
def get_P(m, p):
    p_up, p_down, p_left, p_right=p
    #print(p_up, p_down, p_left, p_right)
    m2 = m ** 2
    P = np.zeros((m2, m2))
    ix_map = {i + 1: (i // m, i % m) for i in range(m2)}
    for i in range(m2):
        for j in range(m2):
            r1, c1 = ix_map[i + 1]
            r2, c2 = ix_map[j + 1]
            rdiff = r1 - r2
            cdiff = c1 - c2
            if rdiff == 0:
                if cdiff == 1:
                    P[i, j] = p_left
                elif cdiff == -1:
                    P[i, j] = p_right
                elif cdiff == 0:
                    if r1 == 0:
                        P[i, j] = p_up
                    elif r1 == m - 1:
                        P[i, j] = p_down
                    if c1 == 0:
                        P[i, j] += p_left
                    elif c1 == m - 1:
                        P[i, j] += p_right
            elif rdiff == 1:
                if cdiff == 0:
                    P[i, j] = p_up
            elif rdiff == -1:
                if cdiff == 0:
                    P[i, j] = p_down
    return P

print("\nTRANSITION MATRIX: 3x3 \n",get_P(m, p))


TRANSITION MATRIX: 3x3 
 [[0.45 0.25 0.   0.3  0.   0.   0.   0.   0.  ]
 [0.25 0.2  0.25 0.   0.3  0.   0.   0.   0.  ]
 [0.   0.25 0.45 0.   0.   0.3  0.   0.   0.  ]
 [0.2  0.   0.   0.25 0.25 0.   0.3  0.   0.  ]
 [0.   0.2  0.   0.25 0.   0.25 0.   0.3  0.  ]
 [0.   0.   0.2  0.   0.25 0.25 0.   0.   0.3 ]
 [0.   0.   0.   0.2  0.   0.   0.55 0.25 0.  ]
 [0.   0.   0.   0.   0.2  0.   0.25 0.3  0.25]
 [0.   0.   0.   0.   0.   0.2  0.   0.25 0.55]]


In [4]:
# THIS SIMPLE CODE JUST SHOWS THE STATE TRANSITION MAPPINGS FOR THE TRANSITION MATRIX
tmp=[]
for i in range(0,m*m):
    tmp2=[]
    for j in range(0,m*m):
        tmp2.append(str(i)+"->"+str(j))
    tmp.append(tmp2)
tmp=np.array(tmp)
print(tmp)

[['0->0' '0->1' '0->2' '0->3' '0->4' '0->5' '0->6' '0->7' '0->8']
 ['1->0' '1->1' '1->2' '1->3' '1->4' '1->5' '1->6' '1->7' '1->8']
 ['2->0' '2->1' '2->2' '2->3' '2->4' '2->5' '2->6' '2->7' '2->8']
 ['3->0' '3->1' '3->2' '3->3' '3->4' '3->5' '3->6' '3->7' '3->8']
 ['4->0' '4->1' '4->2' '4->3' '4->4' '4->5' '4->6' '4->7' '4->8']
 ['5->0' '5->1' '5->2' '5->3' '5->4' '5->5' '5->6' '5->7' '5->8']
 ['6->0' '6->1' '6->2' '6->3' '6->4' '6->5' '6->6' '6->7' '6->8']
 ['7->0' '7->1' '7->2' '7->3' '7->4' '7->5' '7->6' '7->7' '7->8']
 ['8->0' '8->1' '8->2' '8->3' '8->4' '8->5' '8->6' '8->7' '8->8']]


## Code example: numpy

For positive integers n, the power is computed by repeated matrix squarings and matrix multiplications. If n == 0, the identity matrix of the same shape as M is returned. If n < 0, the inverse is computed and then raised to the abs(n).

```
linalg.matrix_power(a, n) --> Raise a square matrix to the (integer) power n.
```

In [5]:
A=np.random.uniform(0,1,(3,3))
print("ORIGINAL:\n",A)
print("\nA**2.0 (component wise square): \n",A**2.0)
print("\nnp.matmul(A,A): \n",np.matmul(A,A))
print("\nnp.linalg.matrix_power(A,2): \n",np.linalg.matrix_power(A,2))


ORIGINAL:
 [[0.39728021 0.18775211 0.3050724 ]
 [0.71696446 0.6222347  0.82518232]
 [0.51930807 0.00771829 0.77566512]]

A**2.0 (component wise square): 
 [[1.57831562e-01 3.52508559e-02 9.30691677e-02]
 [5.14038033e-01 3.87176020e-01 6.80925858e-01]
 [2.69680875e-01 5.95720452e-05 6.01656374e-01]]

np.matmul(A,A): 
 [[0.45086971 0.19377072 0.51276297]
 [1.15947979 0.52815661 1.37224828]
 [0.61465372 0.10829059 0.76645193]]

np.linalg.matrix_power(A,2): 
 [[0.45086971 0.19377072 0.51276297]
 [1.15947979 0.52815661 1.37224828]
 [0.61465372 0.10829059 0.76645193]]


# Assignment

Follow the instructions in the comments 

## Assignment-0:  Simulate the robot

* Lets simulate the broken robot 
* To estimate the steady state occupation probabilities, we can simply simulate the system for a long enough time so that the probabilities eventually converge.  
* The function inputs are 
  * The grid size $m=3$ $\rightarrow$ $m \times m=3 \times 3$
  * The probability $p$ for up,down,left,right moves
  * The initial probabilities distribution vector $q$

In [6]:
# INSERT CODE TO SIMULATE THE BROKEN ROBOT GIVEN THE INPUTS
    # YOU SHOULD ACTUALLY SIMULATE THE PROCESS BY HAVING THE ROBOT "WALK" AROUND 
    # THE SPACE INSIDE A TIME LOOP BASED ON THE p=[pu,pd,pl,pr=0.25]
    # ALSO CODED SO IT "BOUNCES" OFF WHEN IT STEPS INTO A WALL AND RETURNS TO THE ORIGINAL CELL
# THE FUNCTION SHOULD OUTPUT TWO THINGS
    # 1) THE TIME-AVERAGE OCCUPATION PROBABILITY FOR EACH STATE (STORED AS AN m by m MATRIX)
    # 2) THE OCCUPIED STATE AT THE nth TIME-STEP (STORED AS AN m by m MATRIX OF ZEROS AND ONES)

def simulate_robot(n=100000,m=3,q=[0,0,0,0,1,0,0,0,0], p = [0.2, 0.3, 0.25, 0.25]):
    # n=NUMBER OF TIME-STEPS
    # q=INITIALIZATION PROBABILITY: IC MUST BE A VECTOR OF LENGTH (m**2,1) WHICH SUMS TO 1
    # p=[pu=0.2; pd=0.3; pl=0.25; pr=0.25];  

    # INSERT YOUR CODE
    Nn = np.array(q).reshape((m,m))
    Ns = np.array(q).reshape((m,m))
    
    for _ in range(n):
        move = np.random.choice(["Up","Down","Left","Right"], p=p)
        idx = np.where(Nn==1)
        if move == "Up":
            if idx[0]-1 < 0:
                Nn = Nn
            else:
                Nn[idx] = 0
                Nn[idx[0]-1,idx[1]] = 1
                
        elif move =="Down":
            if idx[0]+1 == 3:
                Nn = Nn
            else:
                Nn[idx] = 0
                Nn[idx[0]+1,idx[1]] = 1
                
        elif move == "Left":
            if idx[1]-1 < 0:
                Nn = Nn
            else:
                Nn[idx] = 0    
                Nn[idx[0],idx[1]-1] = 1
                
        elif move == "Right":
            if idx[1]+1 == 3:
                Nn = Nn
            else:
                Nn[idx] = 0
                Nn[idx[0],idx[1]+1] = 1
        Ns = Ns + Nn
    
    
    return [Ns/n,Nn]


## Assignment-1: Compute time average

In [7]:
# TEST YOUR CODE
print("TIME-AVERAGED APPROXIMATE STEADY STATE DISTRIBUTIONS (50000 steps):\n",simulate_robot(m=m,q=q,p=p)[0])

TIME-AVERAGED APPROXIMATE STEADY STATE DISTRIBUTIONS (50000 steps):
 [[0.07071 0.07097 0.07118]
 [0.10511 0.10341 0.10396]
 [0.16165 0.15529 0.15773]]


## Assignment-2: Compute ensemble average

In [10]:
# INSERT CODE TO CALCULATE THE ENSEMBLE AVERAGE OCCUPATION AT THE nth TIME-STEP
# DO THIS BY RUNNING MANY SHORT SIMULATIONS MANY TIMES (e.g. 10000 times)
# THIS CAN BE DONE BY STARTING FROM RANDOM INITIAL CONDITIONS OR BY STARTING FROM THE SAME INITIAL CONDITION EACH TIME
def ensemble_ave(n,q=q):
    ens = np.zeros((3,3))
    for _ in range(10000):
        ens = ens + simulate_robot(n=n,q=q)[1]
    print("SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n =",n,":\n",ens/10000,"\n")


In [11]:
## TEST YOUR CODE (ASSUMING YOU START IN THE CENTER EACH TIME) 

ensemble_ave(1)
#  n=1 expected
#  [[0.   0.2  0.  ]
#  [0.25 0.   0.25]
#  [0.   0.3  0.  ]] 

ensemble_ave(2)
#  n=2 expected
#  [[0.1    0.04   0.1   ]
#  [0.0625 0.245  0.0625]
#  [0.15   0.09   0.15  ]] 

# Notice that at the 20th time step the ensemble average has approximately converge to the time average, since our system is highly ergodic
ensemble_ave(20)

SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n = 1 :
 [[0.     0.2032 0.    ]
 [0.2512 0.     0.2498]
 [0.     0.2958 0.    ]] 

SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n = 2 :
 [[0.0977 0.042  0.102 ]
 [0.0621 0.2422 0.0616]
 [0.1502 0.0925 0.1497]] 

SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n = 20 :
 [[0.0709 0.0716 0.0732]
 [0.1054 0.1011 0.102 ]
 [0.1586 0.1587 0.1585]] 



## Assignment-3:  Matrix multiplication method

- We can also use matrix multiplication to determine state occupation probability $\mathbf q_n$ at the $n^{th}$ time-step: $\mathbf q_n  = \mathbf q_{n-1} \mathbf P$
- **Notice that the matrix multiplication has the effect of statistically progressing the system forward in time. It updates the state occupation probabilities to the next time step, resulting in a new vector, similar to the initial condition probabilities $\mathbf q_0$**
-  We can use this formula recursively, starting from the probability vector $\mathbf q_0$ 
$$\mathbf q_n  = \mathbf q_{0} \mathbf P^n$$
- Where $\mathbf q_0$ is the initial probability distribution and $\mathbf P^n$ is the transition probability matrix applied repeatably with $n$ matrix multiplications
- This is equivalent to the ensemble average discussed above at the $n^{th}$ time step
- Again $\mathbf q_n$ is the probability of the system being in state $i$ after $n$ steps. This is equivalent to the probability of the system transitioning from $\mathbf q_{n-1}$ to $\,\mathbf q_n$

In [12]:
# INSERT CODE TO PREDICT THE OCCUPATION PROBABILITY AT STEP N USING MATRIX MULTIPLICATION
def occupation(q, P, n):
    current = q
    for _ in range(n):
        current = np.dot(current,P)
    return current.reshape(3,3)


In [13]:
## TEST YOUR CODE 
P = get_P(m, p)

# TRY DIFFERENT VALUES OF n
for n in [1,3,20]:
    # print('n = {}\n'.format(n))
    print("\n------- n="+str(n)+" --------")
    ensemble_ave(n,q=q)
    print("\nMATRIX MULTIPLY:")
    print(occupation(q=q, P=P, n=n))


------- n=1 --------
SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n = 1 :
 [[0.     0.1943 0.    ]
 [0.2526 0.     0.2471]
 [0.     0.306  0.    ]] 


MATRIX MULTIPLY:
[[0.   0.2  0.  ]
 [0.25 0.   0.25]
 [0.   0.3  0.  ]]

------- n=3 --------
SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n = 3 :
 [[0.0679 0.1002 0.063 ]
 [0.1366 0.0611 0.1391]
 [0.1231 0.1794 0.1296]] 


MATRIX MULTIPLY:
[[0.0675   0.107    0.0675  ]
 [0.136875 0.06125  0.136875]
 [0.12375  0.1755   0.12375 ]]

------- n=20 --------
SIMULATION ENSEMBLE AVERAGE AT TIMESTEP n = 20 :
 [[0.0671 0.0743 0.0714]
 [0.1061 0.1011 0.1093]
 [0.1575 0.1583 0.1549]] 


MATRIX MULTIPLY:
[[0.07025689 0.07025666 0.07025689]
 [0.10528127 0.10528179 0.10528127]
 [0.15779517 0.15779489 0.15779517]]


## Assignment-4: Eigen-vector method

- There is a third way to calculate the steady-state occupation probability $\mathbf p_n$ as $n \rightarrow \infty$
- The definition of steady state means the the updated $\mathbf q$ vector stops changing with time, i.e. with repeated matrix multiplication.
- mathematically this means $\mathbf q_{n+1}=\mathbf q_n$ can be stated as
$$\mathbf q_{n+1} \mathbf P = \mathbf q_n \rightarrow  \mathbf q_n \mathbf P = \mathbf q_n$$
- This is known as a left-hand eigenvalue problem, which is similar to the right hand eigenvalue problem which you are probably familiar with. You can switch between the two using the matrix transpose!
$$ \mathbf P^T \mathbf q = \mathbf q$$
- for a system with $m^2$ states there will be $m^2$ eigenvalues and $m^2$ eigenvectors.
- One of these eigenvalues will be equal to one, the eigenvector associated with this eigenvalue is the vector of steady-state probabilities
- It is easiest to just see this demonstrated in an example

In [14]:
# INSERT CODE TO COMPUTE THE STEADY STATE OCCUPATION PROBABILITIES 
# FOR THE MARK OFF CHAIN USING THE EIGEN-VALUE METHOD
# SO INCLUDE CODE TO COMPARE IT TO THE MATRIX MULTIPLICATION METHOD WITHIN N=100
w,v = np.linalg.eig(get_P(m,p).T)
print("MATRIX MULTIPLICATION METHOD (n=100) \n",occupation(q=q, P=P, n=n),"\n")
print("EIGEN VALUE/VECTOR METHOD (n=100)")
print("eigen-values: \n",w,"\n")
print("eigen-vector for eval=1: \n",(v[:,1]/v[:,1].sum()).reshape(m,m))

MATRIX MULTIPLICATION METHOD (n=100) 
 [[0.07025689 0.07025666 0.07025689]
 [0.10528127 0.10528179 0.10528127]
 [0.15779517 0.15779489 0.15779517]] 

EIGEN VALUE/VECTOR METHOD (n=100)
eigen-values: 
 [-0.49494897  1.          0.75        0.74494897  0.49494897 -0.00505103
  0.00505103  0.25505103  0.25      ] 

eigen-vector for eval=1: 
 [[0.07017544 0.07017544 0.07017544]
 [0.10526316 0.10526316 0.10526316]
 [0.15789474 0.15789474 0.15789474]]
